In [69]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

In [79]:
info = pd.read_csv("export_2024-10-2.csv")

pdf_files = list(Path("export_2024-10-2").iterdir())

pdf_table = pd.DataFrame({
    "year": [pdf_file.name.split("_")[0] if re.match("[0-9]+", pdf_file.name.split("_")[0]) else "NaN" for pdf_file in pdf_files],
    "pmid": [float(pdf_file.name.split("_")[-1].replace(".pdf", "")) if re.match("[0-9]+", pdf_file.name.split("_")[-1]) else np.nan for pdf_file in pdf_files],
    "path": [pdf_file.resolve() for pdf_file in pdf_files],
    "title": [pdf_file.name.split("_")[3] if len(pdf_file.name.split("_")) > 1 else "NaN" for pdf_file in pdf_files],
    "first_auth": [pdf_file.name.split("_")[1] if len(pdf_file.name.split("_")) > 1 else "NaN" for pdf_file in pdf_files],
    "last_auth": [pdf_file.name.split("_")[2] if len(pdf_file.name.split("_")) > 1 else "NaN" for pdf_file in pdf_files],
    "name": [file.name for file in pdf_files]
})

In [80]:
info["file_location"] = ["" for _ in range(len(info))]
info["match_via"] = ["" for _ in range(len(info))]
for row_idx in range(info.shape[0]):
    title_oi = info["title"][row_idx].replace(":", "-").replace("/", "-")
    first_auth = info["author"][row_idx].split(",")[0]
    last_auth = info["author"][row_idx].split(",")[-1]
    year = info["year"][row_idx]
    pmid = info["pmid"][row_idx]

    # match via pmid
    if not np.isnan(pmid) and pmid in pdf_table["pmid"].to_list():

        for pdf_idx in range(pdf_table.shape[0]):
            pmid_pdf = pdf_table["pmid"][pdf_idx]
            file_location = pdf_table["path"][pdf_idx]

            if not np.isnan(pmid_pdf):
                if np.isclose(pmid, pmid_pdf):
                    info["file_location"].values[row_idx] = file_location
                    info["match_via"].values[row_idx] = "pmid"

    # match via title
    else:
        for pdf_idx in range(pdf_table.shape[0]):
            pmid_pdf = pdf_table["pmid"][pdf_idx]
            file_location = pdf_table["path"][pdf_idx]
            pdf_name = pdf_table["name"][pdf_idx]

            if title_oi in pdf_name:
                info["file_location"].values[row_idx] = file_location
                info["match_via"].values[row_idx] = "title"

In [82]:
info_subset = info[["title", "doi", "file_location"]].query("file_location != ''")
info_subset.to_csv("manifest.csv")
info_subset

,title,doi,file_location
0,Spatial cell type mapping of multiple sclerosi...,10.1101/2022.11.03.514906,/Users/pschafer/Projects/MS_expert/export_2024...
1,Multiple sclerosis – a review,10.1111/ene.13819,/Users/pschafer/Projects/MS_expert/export_2024...
2,Multiple Sclerosis,10.1056/nejmra1401483,/Users/pschafer/Projects/MS_expert/export_2024...
3,Mendelian randomization shows a causal effect ...,10.1212/nxg.0000000000000097,/Users/pschafer/Projects/MS_expert/export_2024...
4,Diversity and Function of Glial Cell Types in ...,10.1016/j.it.2021.01.005,/Users/pschafer/Projects/MS_expert/export_2024...
5,A lymphocyte–microglia–astrocyte axis in chron...,10.1038/s41586-021-03892-7,/Users/pschafer/Projects/MS_expert/export_2024...
6,Repeated subcutaneous injections of IL12/23 p4...,10.1016/s1474-4422(08)70173-x,/Users/pschafer/Projects/MS_expert/export_2024...
7,Lesion genesis in a subset of patients with mu...,10.1093/brain/awm236,/Users/pschafer/Projects/MS_expert/export_2024...
8,Prediction of combination therapies based on t...,10.1186/s13073-021-00925-8,/Users/pschafer/Projects/MS_expert/export_2024...
9,Longitudinal analysis reveals high prevalence ...,10.1126/science.abj8222,/Users/pschafer/Projects/MS_expert/export_2024...
